In [4]:
import pandas as pd
import cv2 


df = pd.read_csv("/Users/menghang/Desktop/ml-dev/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/df2.txt")

In [5]:
ball = df[df["label"] == "ball"]

In [6]:
ball_possession = []

In [ ]:
for i, row in ball.iterrows():
    frame = df[df["frame_id"] == row["frame_id"]]

    ball_center = [row["x_homo"], row["y_homo"]]   
    players_centers = []
    for j, f in frame.iterrows(): 
        if(f["label"] != "ball"):
         players_centers.append([f["x_homo"], f["y_homo"]])
    
    player, index = find_nearest_coordinate(np.array(ball_center), np.array(players_centers))
    b = frame.iloc[index]
    while b.color == "red" or b.color == "green":
        index += 1
        b = frame.iloc[index]
    
    print(b.color)
    
    if i == 30:
       break
    ball_possession.append(b["color"])
                          
                          
                                        

In [124]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 


blue = cv2.VideoCapture("demo_video/blue.mp4")
white = cv2.VideoCapture("demo_video/white.mp4")


writer = cv2.VideoWriter("blue.mp4", cv2.VideoWriter_fourcc(*'MJPG'), 25, (848, 1229) )




while True:

    b, bl = blue.read()
    # w, wh = white.read()

    if b:

        bl = bl[536:536+848,46: 46 + 1229]
        # wh = wh[536:536+848,46: 46 + 1229]
        bl = cv2.rotate(bl ,cv2.ROTATE_90_CLOCKWISE)
        # break
    
    
        

        cv2.imshow("frame", bl)
      
        
        # key = cv2.waitKey(0)
        # if key == ord('q'):
        #     cv2.destroyAllWindows()
        #     cv2.waitKey(1)
        #     break
        writer.write(bl)
    else:
        break
    

writer.release()




: 

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 

detection = cv2.VideoCapture("demo_video/concat.avi")
possession = cv2.VideoCapture("demo_video/ball_possession.mp4")
template = cv2.VideoCapture("demo_video/template_2d.avi")
blue = cv2.VideoCapture("demo_video/blue.mp4")
white = cv2.VideoCapture("demo_video/white.mp4")


writer = cv2.VideoWriter("video.mp4", cv2.VideoWriter_fourcc(*'MJPG'), 25, (1060, 2160) )




while True:
    d, det = detection.read()
    p, po = possession.read()
    t, tp = template.read()
    b, bl = blue.read()
    w, wh = white.read()

    if d and p and t and b and w:

        bl = bl[536:536+848,46: 46 + 1229]
        wh = wh[536:536+848,46: 46 + 1229]

        heatmap = np.hstack([bl, wh])
        shape = (1060, 540)
        det = cv2.resize(det, shape, interpolation=cv2.INTER_AREA)
        po =  cv2.resize(po, shape, interpolation=cv2.INTER_AREA)
        tp =  cv2.resize(tp, shape, interpolation=cv2.INTER_AREA)
        heatmap =  cv2.resize(heatmap, shape, interpolation=cv2.INTER_AREA)
        
        vstck = np.vstack([det, tp, po , heatmap])

        cv2.imshow("frame", vstck)
        # key = cv2.waitKey(0)
        # if key == ord('q'):
        #     cv2.destroyAllWindows()
        #     cv2.waitKey(1)
        #     break
        writer.write(vstck)
    else:
        break
    

writer.release()




KeyboardInterrupt: 

In [2]:
import cv2
import numpy as np
import math
def draw_possession_bar(image, possession_percentage, bar_height=30, padding=20, border_radius=5):
    # Calculate rectangle dimensions based on possession percentage
    total_bar_width = image.shape[1] - 2 * padding
    width = int(total_bar_width * possession_percentage / 100)

    # Draw the bar background
    cv2.rectangle(image, (padding, padding + 80), (image.shape[1] - padding, padding + bar_height + 80), (8,20,223), -1)
    # Draw the blue side of the bar with rounded corners
    cv2.rectangle(image, (padding, padding + 80), (padding + width, padding + bar_height + 80),(8,122,244) , -1)

    image = cv2.putText(image, str(math.ceil(possession_percentage))+"%", (padding, padding + 60), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 
                   0.7, (255,255,255), 2, cv2.LINE_AA)
    image = cv2.putText(image, str(math.floor(100 - possession_percentage))+"%", (padding + 510, padding + 60), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 
                   0.7, (255,255,255), 2, cv2.LINE_AA)
                   


    return image

# Create a blank image
image = np.zeros((300, 600, 3), dtype=np.uint8)
image[:,: ,0] = 54
image[:,: ,1] = 39 
image[:,: ,2] = 36



# Set possession percentage (change this value to see the effect)
possession_percentage = 28.99

# Draw the possession bar on the image with padding and rounded corners
image_with_bar = draw_possession_bar(image.copy(), possession_percentage, bar_height=40, padding=20, border_radius=10)

# Display the image with the possession bar
cv2.imshow('Ball Possession', image_with_bar)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)



-1

In [7]:
import pandas as pd
import numpy as np 
import cv2
def find_nearest_coordinate(point, coordinates):
    distances = np.linalg.norm(coordinates - point, axis=1)  # Calculate Euclidean distances
    nearest_index = np.argmin(distances)  # Find index of the minimum distance
    nearest_coordinate = coordinates[nearest_index]  # Get the nearest coordinate
    return nearest_coordinate, nearest_index
df = pd.read_csv("/Users/menghang/Desktop/ml-dev/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/df2.txt")

ball = df[df["label"] == "ball"]
ball_possession = []


out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 25, (200,400))

ball_frame = list(ball["frame_id"])

frame_count = 0


ball_found = 0

possession = {"white": 0, "blue": 0 }


for i, row in ball.iterrows():
    frame = df[df["frame_id"] == row["frame_id"]]

    ball_center = [row["x_homo"], row["y_homo"]]   
    players_centers = []
    for j, f in frame.iterrows(): 
        if(f["label"] != "ball"):
         players_centers.append([f["x_homo"], f["y_homo"]])
    
    player, index = find_nearest_coordinate(np.array(ball_center), np.array(players_centers))
    b = frame.iloc[index]
    while b.color == "red" or b.color == "green":
        index += 1
        b = frame.iloc[index]
    
 
    ball_possession.append(b["color"])
                          
occrence = {"white" : 0, "blue":0, "red": 0, "green": 0}
for i in ball_possession:
    occrence[i] += 1





In [9]:

cap = cv2.VideoCapture("/Users/menghang/Desktop/ml-dev/football-server/video_demo/concat.avi")

out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 25, (600,300))

ball_frame = list(ball["frame_id"])

frame_count = 0


ball_found = 0

possession = {"white": 0, "blue": 0 }
image = np.zeros((300, 600, 3), dtype=np.uint8)
image[:,: ,0] = 54
image[:,: ,1] = 39 
image[:,: ,2] = 36


while True:     
    success, frame = cap.read()
    if success:
        if frame_count in ball_frame: 
            ball_found += 1
            possession[ball_possession[ball_found-1] ] += 1 
            home = round(possession["blue"] / ball_found, 2)    
            image_with_bar = draw_possession_bar(image.copy(), home * 100, bar_height=40, padding=20, border_radius=10)
            print(image_with_bar.shape)
            out.write(image_with_bar)

        
        if ball_found == 0:
            image_with_bar = draw_possession_bar(image.copy(), 50, bar_height=40, padding=20, border_radius=10)
           
            out.write(image_with_bar)
        else:
            out.write(image_with_bar)
        frame_count += 1

     
    
  
    
    else:
        break
out.release()
cap.release()
cv2.destroyAllWindows()


(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 600, 3)
(300, 